# Import packages

In [2]:
import numpy as np
import cvxpy as cvx
import pandas as pd
import logging
from functools import partial
import holoviews as hv
import sys
import holoviews as hv
from hw10_script import *
hv.extension('bokeh')

Output hidden; open in https://colab.research.google.com to view.

# Custom functions

In [13]:
import scipy.sparse
import numpy as np

from cvxpy.expressions.expression import Expression
from cvxpy.atoms.affine.reshape import reshape
from cvxpy.atoms.affine.sum import sum as sum_
from cvxpy.atoms.elementwise.maximum import maximum


def log_normcdf(x):
    """Elementwise log of the cumulative distribution function of a standard normal random variable.
    Implementation is a quadratic approximation with modest accuracy over [-4, 4].
    """
    A = scipy.sparse.diags(
        np.sqrt(
            [
                0.02301291,
                0.08070214,
                0.16411522,
                0.09003495,
                0.08200854,
                0.01371543,
                0.04641081,
            ]
        )
    )
    b = np.array([[3.0, 2.0, 1.0, 0.0, -1.0, -2.5, -3.5]]).reshape(-1, 1)

    x = Expression.cast_to_const(x)
    flat_x = reshape(x, (1, x.size))

    y = A @ (b @ np.ones(flat_x.shape) - np.ones(b.shape) @ flat_x)
    out = -sum_(maximum(y, 0) ** 2, axis=0)

    return reshape(out, x.shape)

# Question 1: Maximum likelihood prediction of team ability

Python scripts need to be updated to the runtime of this Google Colab!

In [3]:
n, m, sigma, m_test, train, test, A_train, A_test = synt_team_data()

In [14]:
alpha = cvx.Variable(shape = (n,1), nonneg=True)
a = np.dot(A_train , alpha) * (1/sigma)     # normalizing s.t the probability will be <=1
#ll = cvx_log_noramcdf(a) # function does not support cvxpy input :( (USE MATLAB OR SUGGEST PR)
ll = log_normcdf(a)
obj_func = cvx.Maximize(cvx.sum(ll))
constraints = [alpha<=1,
               ] 
prob = cvx.Problem(obj_func ,constraints)
G_star = prob.solve()

In [19]:
print(alpha.value)
# This is correct!

[[1.        ]
 [0.        ]
 [0.67827377]
 [0.36870462]
 [0.79002166]
 [0.58130641]
 [0.38738245]
 [0.08544141]
 [0.67004625]
 [0.57748618]]


# Question 2: Allocation of interdiction effort

In [20]:
n, m, a, x_max, B, edges, A = synt_interdict_alloc_data()

In [21]:
z = cvx.Variable((n, 1))
x = cvx.Variable((m, 1), nonneg=True)
obj_func = cvx.Minimize(z[-1])
constraints = [x<=x_max,
               cvx.sum(x)<=B,
               z[0] == 0, 
               A.T*z >= -np.diag(a.squeeze()) * x
               ]
prob = cvx.Problem(obj_func, constraints)
logPn = prob.solve()
np.exp(logPn)

0.054386168982720626

Uniform allocation

In [22]:
z = cvx.Variable((n, 1))
x = (B/m) * np.ones((m,1))
obj_func = cvx.Minimize(z[-1])
constraints = [z[0] == 0, 
              A.T*z >= (-np.diag(a.squeeze()).dot(x.squeeze()))[:,None]
               ]
prob = cvx.Problem(obj_func, constraints)
logPn = prob.solve()
np.exp(logPn)

0.36552401685031166